## CS3110 Final Project

Jack Giuliani & Alex Shapovalov

Write description of the project here

In [4]:
# Base code

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

def laplace_mech(v, sensitivity, epsilon):
    return v + np.random.laplace(loc=0, scale=sensitivity / epsilon)

def gaussian_mech(v, sensitivity, epsilon, delta):
    return v + np.random.normal(loc=0, scale=sensitivity * np.sqrt(2*np.log(1.25/delta)) / epsilon)

def is_k_anonymous(k, qis, df):
    return df[qis].value_counts().min() >= k

data = pd.read_csv('patients.csv')

In [6]:
# 1. Laplace for healthcare expenses and healhcare coverage here:

In [ ]:
# 2. Guassian for healthcare expenses and healthcare coverage here:

In [ ]:
# 3. Comparison between laplace and gaussian

In [ ]:
# 4. Create k-anonymous address data here

In [ ]:
# 5. Perform a linkage attack

In [79]:
data_pii = data[['FIRST', 'LAST', 'BIRTHDATE', 'CITY', 'STATE', 'ZIP']]
data_pii.head()

,FIRST,LAST,BIRTHDATE,CITY,STATE,ZIP
0,Damon455,Langosh790,2019-02-17,Springfield,Massachusetts,1104.0
1,Thi53,Wunsch504,2005-07-04,Bellingham,Massachusetts,NaN
2,Chi716,Greenfelder433,1998-05-11,Boston,Massachusetts,2131.0
3,Phillis443,Walter473,2003-01-28,Hingham,Massachusetts,2043.0
4,Jerrold404,Herzog843,1993-12-23,Revere,Massachusetts,NaN


In [80]:
data_deid = data.drop(columns=['Id', 'DEATHDATE', 'SSN', 'DRIVERS', 'PASSPORT', 'PREFIX', 'FIRST', 'LAST', 'SUFFIX', 'MAIDEN', 'BIRTHPLACE', 'ADDRESS', 'LAT', 'LON'])
data_deid.head()

,BIRTHDATE,MARITAL,RACE,ETHNICITY,GENDER,CITY,STATE,COUNTY,ZIP,HEALTHCARE_EXPENSES,HEALTHCARE_COVERAGE
0,2019-02-17,NaN,white,nonhispanic,M,Springfield,Massachusetts,Hampden County,1104.0,9039.1645,7964.1255
1,2005-07-04,NaN,white,nonhispanic,F,Bellingham,Massachusetts,Norfolk County,NaN,402723.4150,14064.1350
2,1998-05-11,NaN,white,nonhispanic,M,Boston,Massachusetts,Suffolk County,2131.0,571935.8725,787.5375
3,2003-01-28,NaN,white,nonhispanic,F,Hingham,Massachusetts,Plymouth County,2043.0,582557.8030,104782.2070
4,1993-12-23,M,black,nonhispanic,M,Revere,Massachusetts,Suffolk County,NaN,475826.8550,18067.0950


In [81]:
def linking_attack(cols):
    data_linked = pd.merge(data_pii, data_deid, left_on=cols, right_on=cols)
    return data_linked

print(len(linking_attack(['ZIP'])))
print(len(linking_attack(['BIRTHDATE'])))
print(len(linking_attack(['ZIP', 'BIRTHDATE'])))
print(len(linking_attack(['ZIP', 'BIRTHDATE', 'CITY'])))

299659
1851
1533
1531


In [ ]:
# 6. Clipping on expenses column

In [52]:
def find_b():
    bs = range(1, 5000000, 100000)
    epsilon_i = 1/len(bs)
    old_result = 0
    for b in bs:
        result = data['HEALTHCARE_EXPENSES'].clip(upper=b).sum()
        dp_result = laplace_mech(result, sensitivity=b, epsilon=epsilon_i)
        print(b, dp_result)
        
find_b()

1 1096.8265362898844
100001 119998899.80820711
200001 216039058.89789757
300001 357640765.3337801
400001 420332769.86732775
500001 482599487.91595674
600001 578936645.8743817
700001 696418121.7253163
800001 688506128.7183858
900001 774248384.8088973
1000001 818534976.8439758
1100001 936227484.2993749
1200001 925851489.4894291
1300001 1109396880.126967
1400001 1145606760.6692567
1500001 1239917830.7214274
1600001 1197400829.3010738
1700001 1222429678.8867998
1800001 1226149817.0378456
1900001 1146300979.5329332
2000001 1375117669.7701068
2100001 1283815648.8167098
2200001 1345041131.2313614
2300001 1845192377.7560043
2400001 1295996988.397218
2500001 1414622117.92941
2600001 1175789813.5654337
2700001 1167724283.529558
2800001 1100118239.940289
2900001 1522705432.209178
3000001 1436423975.3751583
3100001 1480422855.775668
3200001 1394232814.5324576
3300001 1519648074.0946996
3400001 1541726404.3326366
3500001 1555599521.1804986
3600001 1275496936.6188238
3700001 1985863102.9272218
38000

In [53]:
def mean_expenses_clipping(epsilon):
    df = data['HEALTHCARE_EXPENSES']
    lower = 0
    upper = 2900001

    dp_sum = laplace_mech(df.clip(upper=upper).sum(), sensitivity=upper, epsilon=epsilon/2)
    dp_count = laplace_mech(len(df), sensitivity=1, epsilon=epsilon/2)
    return dp_sum/dp_count

mean_expenses_clipping(1.0)

1217545.8183148874

In [ ]:
# 7. Propose test release vs SAA

In [62]:
def ls_mean_at_distance(df, b, k):
    n = len(df)
    return b/(n - k)

def dist_to_high_ls_mean(df, b, s):
    k = 0
    while ls_mean_at_distance(df, b, k) < s:
        k += 1
    return k

def mean_expenses_ptr(s, epsilon, delta):
    df = data['HEALTHCARE_EXPENSES']
    lower = 0
    upper = 2900001
    
    distance = dist_to_high_ls_mean(df, b, s)
    noisy_distance = laplace_mech(distance, sensitivity=1, epsilon=epsilon)
    test_result = noisy_distance < np.log(2/delta)/(2*epsilon)
    if test_result:
        return False

    result = df.clip(lower=0, upper=b).mean()
    return laplace_mech(result, sensitivity=s, epsilon=epsilon)

mean_ptr(10000, 1, 1e-5) # fix with better value for s

178063.71994518308

In [55]:
def f(chunk):
    return chunk.mean()

def mean_expenses_saa(k, epsilon):
    df = data['HEALTHCARE_EXPENSES']
    lower = 0
    upper = 2900001 # replace with clipping parameter in 6
    
    chunks = np.array_split(df, k)
    answers = [f(chunk) for chunk in chunks]
    answers_clipped = pd.Series(answers).clip(lower=lower, upper=upper)
    answers_clipped_avg = answers_clipped.mean()
    return laplace_mech(answers_clipped_avg, sensitivity=upper/k, epsilon=epsilon)

mean_expenses_saa(20, 1.0)

C:\Users\jackg\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)


1228961.9707853706

In [ ]:
# 8. Gradient descent